In [109]:
import pandas as pd
import numpy as np
#import math
import datetime as dt
from pandas.tseries.offsets import DateOffset

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

#import seaborn as sns 
import matplotlib.pyplot as plt 
#from colorama import Fore

df = pd.read_csv('./data/AAPL.csv')
#training_set = df.iloc[:1007, 1:2].values
#test_set = df.iloc[1007:, 1:2].values
df1 = df.rename(columns={'Date': 'date','Open':'open','High':'high','Low':'low','Close':'close',
                                'Adj Close':'adj_close','Volume':'volume'})
df1.head(5)
closedf = df1[['date','close']]
# print("Shape of close dataframe:", closedf.shape)
closedf = closedf[closedf['date'] > '2020-09-13']
close_stock = closedf.copy()
del closedf['date']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
# print(closedf.shape)
training_size=int(len(closedf)*0.70)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
# print("train_data: ", train_data.shape)
# print("test_data: ", test_data.shape)
# fig, ax = plt.subplots(figsize=(15, 6))
# sns.lineplot(x = close_stock['date'][:255], y = close_stock['close'][:255], color = 'black')
# sns.lineplot(x = close_stock['date'][255:], y = close_stock['close'][255:], color = 'red')

# Formatting
#ax.set_title('Train & Test data', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
#ax.set_xlabel('Date', fontsize = 16, fontdict=dict(weight='bold'))
#ax.set_ylabel('Weekly Sales', fontsize = 16, fontdict=dict(weight='bold'))
#plt.tick_params(axis='y', which='major', labelsize=16)
#plt.tick_params(axis='x', which='major', labelsize=16)
#plt.legend(loc='upper right' ,labels = ('train', 'test'))
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# print("X_train: ", X_train.shape)
# print("y_train: ", y_train.shape)
# print("X_test: ", X_test.shape)
# print("y_test", y_test.shape)

#!pip install xgboost
from xgboost import XGBRegressor
my_model = XGBRegressor(n_estimators=1000)
my_model.fit(X_train, y_train, verbose=False)
predictions = my_model.predict(X_test)
#print("Mean Absolute Error - MAE : " + str(mean_absolute_error(y_test, predictions)))
#print("Root Mean squared Error - RMSE : " + str(math.sqrt(mean_squared_error(y_test, predictions))))
train_predict=my_model.predict(X_train)
test_predict=my_model.predict(X_test)

train_predict = train_predict.reshape(-1,1)
test_predict = test_predict.reshape(-1,1)

print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
#print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
##print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])

plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

# fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
#                                           plotdf['test_predicted_close']],
#               labels={'value':'Close price','date': 'Date'})
# fig.update_layout(title_text='Comparision between original close price vs predicted close price',
#                   plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
# fig.for_each_trace(lambda t:  t.update(name = next(names)))

# fig.update_xaxes(showgrid=False)
# fig.update_yaxes(showgrid=False)
# fig.show()

x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 30
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        
        yhat = my_model.predict(x_input)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat.tolist())
        temp_input=temp_input[1:]
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        yhat = my_model.predict(x_input)
        
        temp_input.extend(yhat.tolist())
        lst_output.extend(yhat.tolist())
        
        i=i+1
        
#print("Output of predicted next days: ", len(lst_output))
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
# print(last_days)
# print(day_pred)
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

# fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
#                                                       new_pred_plot['next_predicted_days_value']],
#               labels={'value': 'Close price','index': 'Timestamp'})
# fig.update_layout(title_text='Compare last 15 days vs next 10 days',
#                   plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
# fig.for_each_trace(lambda t:  t.update(name = next(names)))
# fig.update_xaxes(showgrid=False)
# fig.update_yaxes(showgrid=False)
# fig.show()
plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

# fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
#                                           plotdf['test_predicted_close']],
#               labels={'value':'Close price','date': 'Date'})
# fig.update_layout(title_text='Comparision between original close price vs predicted close price',
#                   plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
# fig.for_each_trace(lambda t:  t.update(name = next(names)))

# fig.update_xaxes(showgrid=False)
# fig.update_yaxes(showgrid=False)
# fig.show()

my_model=closedf.tolist()
my_model.extend((np.array(lst_output).reshape(-1,1)).tolist())
my_model=scaler.inverse_transform(my_model).reshape(1,-1).tolist()[0]

names = cycle(['Close Price'])

# fig = px.line(my_model,x=plotdf['date'],labels={'value': 'Close price','index': 'Date'})
# fig.update_layout(title_text='Plotting whole closing price with prediction',
#                   plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')
# fig.for_each_trace(lambda t:  t.update(name = next(names)))
# fig.update_xaxes(showgrid=False)
# fig.update_yaxes(showgrid=False)
# fig.show()

# Number of previous days' data used for prediction
n_input = 60

# Get the last `n_input` days from the training data set
last_days = training_set_scaled[-n_input:]

# Reshape to match the input shape of the model
last_days_scaled = np.reshape(last_days, (1, n_input, 1))

# Number of future days to predict
n_future = 30  # e.g. predict the next month

# List to hold the predictions
predictions = []

for _ in range(n_future):
    # Predict the next day
    predicted_value = model.predict(last_days_scaled)
    
    # Reshape predicted_value to match the dimensions of last_days_scaled
    predicted_value = np.reshape(predicted_value, (1, 1, 1))
    
    # Append the predicted value to the list of predictions
    predictions.append(sc.inverse_transform(predicted_value[0, 0, 0].reshape(-1, 1))[0][0])    # Append the predicted value to the last_days_scaled and remove the first value
    last_days_scaled = np.append(last_days_scaled[:, 1:, :], predicted_value, axis=1)

# Create a DataFrame with the future dates and predicted prices
future_dates = pd.date_range(start='2022-03-01', periods=n_future, freq='B')  # Adjust the start date as needed
predicted_prices = np.array(predictions)

prediction_df = pd.DataFrame(data={'Date': future_dates, 'Predicted Close': predicted_prices})

print(prediction_df)
# Create a trace for the predicted prices
trace1 = go.Scatter(
    x=prediction_df['Date'],
    y=prediction_df['Predicted Close'],
    mode='lines',
    name='Predicted Close'
)

data = [trace1]

layout = go.Layout(
    title="Predicted Stock Prices",
    xaxis={'title': "Date"},
    yaxis={'title': "Close Price"}
)

fig = go.Figure(data=data, layout=layout)

fig.show()

Train data prediction: (588, 1)
Test data prediction: (244, 1)
1/1 [==============================] - 0s 9ms/step
         Date  Predicted Close
0  2022-03-01       130.327438
1  2022-03-02       129.503479
2  2022-03-03       129.276306
3  2022-03-04       127.094543
4  2022-03-07       119.795494
5  2022-03-08       118.793938
6  2022-03-09       117.254593
7  2022-03-10       115.368210
8  2022-03-11       111.672409
9  2022-03-14       110.464546
10 2022-03-15       108.834915
11 2022-03-16       106.746437
12 2022-03-17       104.375549
13 2022-03-18       102.517319
14 2022-03-21       100.830605
15 2022-03-22        98.687073
16 2022-03-23        96.854469
17 2022-03-24        95.192230
18 2022-03-25        93.896339
19 2022-03-28        92.053337
20 2022-03-29        90.702507
21 2022-03-30        88.838913
22 2022-03-31        87.786537
23 2022-04-01        86.050247
24 2022-04-04        85.176231
25 2022-04-05        83.157059
26 2022-04-06        82.603973
27 2022-04-07     